In [3]:

def read_cacm():
    file_name = "./Data/CACM/cacm.all"
    file = open(file_name)
    documents = []
    for line in file.readlines():
        curent_section = -1
        if line[0] == '.':
            if line[1] == 'I':
                documents.append(["", "", ""])
            elif line[1] == 'T':
                current_section = 0
            elif line[1] == 'W':
                current_section = 1
            elif line[1] == 'K':
                current_section = 2
            else:
                current_section = -1
        else:
            if current_section >= 0:
                if len(documents[-1][current_section]) == 0:
                    documents[-1][current_section] += line.strip('\n')
                else:
                    documents[-1][current_section] += " " + line.strip('\n')
    file.close()  
    return documents

In [4]:
documents = read_cacm()
print(documents[1654])


['Code Extension Procedures for Information Interchange* (Proposed USA Standard)', '', 'standard code, code, information interchange, characters, shift out, shift in, escape, data link  escape, control functions, standard procedures, code extension, code table, bit pattern']


In [5]:
def read_forbidden_words():
    file_name = "./Data/CACM/common_words"
    file = open(file_name)
    words = []
    for word in file.readlines():
        words.append(word.strip().lower())
    file.close()
    return words

In [6]:
import nltk

def tokenize(documents):
    d = {}
    forbidden_words = read_forbidden_words()
    for i in range(len(documents)):
        document = documents[i]
        for text in document:
            if len(text) > 0:
                tokens = nltk.word_tokenize(text)
                for token in tokens:
                    if token.lower() not in forbidden_words:
                        if token in d:
                            d[token].append(i+1)
                        else:
                            d[token] = [i+1]
    return d

In [7]:
tokenize(documents)

{'Preliminary': [1,
  1726,
  1771,
  1946,
  2050,
  2163,
  2181,
  2389,
  2398,
  2718,
  2929,
  2972],
 'Report-International': [1],
 'Algebraic': [1,
  21,
  54,
  55,
  99,
  284,
  393,
  1214,
  1394,
  1397,
  1397,
  1397,
  2090,
  2165,
  2166,
  2167,
  2167,
  3189,
  3199,
  3203],
 'Language': [1,
  54,
  82,
  93,
  99,
  196,
  208,
  265,
  292,
  300,
  407,
  584,
  616,
  616,
  644,
  654,
  655,
  658,
  681,
  691,
  763,
  795,
  800,
  829,
  830,
  889,
  892,
  982,
  987,
  1035,
  1043,
  1086,
  1134,
  1135,
  1149,
  1162,
  1166,
  1167,
  1178,
  1179,
  1180,
  1184,
  1188,
  1244,
  1253,
  1262,
  1272,
  1336,
  1380,
  1389,
  1395,
  1454,
  1455,
  1483,
  1484,
  1487,
  1519,
  1541,
  1542,
  1602,
  1680,
  1681,
  1681,
  1693,
  1700,
  1767,
  1824,
  1835,
  1856,
  1856,
  1856,
  1860,
  1865,
  1989,
  2059,
  2060,
  2091,
  2091,
  2092,
  2126,
  2127,
  2138,
  2178,
  2230,
  2246,
  2264,
  2301,
  2301,
  2310,
  2406,
  2

In [23]:
d = tokenize(documents)
print("Nombre de tokens :")
T1 = sum([len(L) for L in d])
print(T1)
print("Vocabulaire : ")
M1 = len(d)
print(M1)

Nombre de tokens :
116984
Vocabulaire : 
13968


In [24]:
def read_half_cacm():
    documents = read_cacm()
    return documents[:len(documents)//2]


In [25]:
d_half = tokenize(read_half_cacm())
print("Nombre de tokens :")
T2 = sum([len(L) for L in d_half])
print(T2)
print("Vocabulaire : ")
M2 = len(d_half)
print(M2)

Nombre de tokens :
59313
Vocabulaire : 
7278


In [27]:
from math import log, pow
b = log(M1/M2)/log(T1/T2)
k = M1/(pow(T1, b))
print("K = {}, b = {}".format(k, b))

K = 0.19084623884743412, b = 0.9598123986549654


In [29]:
print('Pour 1 million de tokens, vocabulaire :')
print(int(k * pow(1e6, b))

SyntaxError: unexpected EOF while parsing (<ipython-input-29-b337d084b40d>, line 2)